In [ ]:
import json
import os
import random
import fnmatch
import openpyxl

from urllib.request import urlopen
import urllib

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import re

In [ ]:
sns.set_style("darkgrid")

In [ ]:
def import_json_to_dict(url) :
    response = urllib.request.urlopen(url)
    my_dict = json.loads(response.read())
    return my_dict

In [ ]:
def clean_str(s):
    d = {
        "’": "'",
        "\xa0": " ",
        "/":","
        }
    for x in d:
        s = s.replace(x, d[x]).strip()
    return s

In [ ]:
month_dict = {
    "January" : "Janvier",
"February": "Février",
"March": "Mars",
"April" : "Avril",
"May" : "Mai",
"June" : "Juin",
"July" : "Juillet",
"August" : "Août",
"September":"Septembre",
"October":"Octobre",
"November":"Novembre",
"December":"Décembre"}

In [ ]:
data_dir_path = './data/'

In [ ]:
mailles_list = ['national', 'regional', 'departemental']

In [ ]:
propilot_path = os.path.join("data")

In [ ]:
df_dict = {}
file_list = [
"dim_activity",
"dim_tree_nodes",
"dim_top_levels",
"dim_maturities",
"dim_period",
"dim_snapshots",
"dim_effects",
"dim_properties",
"dim_states",
"dim_structures",
"fact_financials",
"fact_property_values"
]

In [ ]:
for file in file_list:
    for file_csv in os.listdir(propilot_path):
        if fnmatch.fnmatch(file_csv, file + "*.csv"):
            print("File loaded : ", file_csv)
            df_dict[file] = pd.read_csv(os.path.join(propilot_path, file_csv), sep=";")

In [ ]:
df = (df_dict["fact_financials"]
      .merge(df_dict["dim_tree_nodes"], left_on="tree_node_id", right_on="tree_node_id")
      .merge(df_dict["dim_effects"], left_on="effect_id", right_on="effect_id")
      .merge(df_dict["dim_states"], left_on="state_id", right_on="state_id")
      .merge(df_dict["dim_period"], left_on="period_id", right_on="period_id")
      .merge(df_dict["dim_structures"], left_on="structure_id", right_on="structure_id"))

In [ ]:
def extract_dep_code(expr):
    nums = re.findall(r'D\d+', expr)
    if expr.endswith('D2A'):
        return '2A'
    elif expr.endswith('D2B'):
        return '2B'
    elif expr.endswith('E00'):
        return '999'
    return nums[0][1:].zfill(2) if len(nums) > 0 else None

df['dep_code'] = df['tree_node_code'].apply(lambda x: extract_dep_code(x))

In [ ]:
cols = ["tree_node_name", "structure_name", "effect_id", "state_id", "period_date", "period_month_tri", "period_month_year", "financials_cumulated_amount", "dep_code"]
df=df[cols]
df.rename(columns={"period_month_year":"Date", "financials_cumulated_amount":"valeur"}, inplace=True)

In [ ]:
forbidden_period_value = ["Y", "Total"]

In [ ]:
df_dep = df.loc[
    (df.structure_name == "Département") & 
    (~df.period_month_tri.isin(forbidden_period_value)) &
    (df.state_id == 'Valeur Actuelle') &
    (~df.valeur.isna())].copy()

In [ ]:
df_dep["departement"] = df_dep["tree_node_name"].apply(lambda x: x.split("/")[0].strip())
df_dep["mesure"] = df_dep["tree_node_name"].apply(lambda x: x.split("/")[1].strip())

# nettoyage de la colonne mesure, on enlève un point surnuméraire.
df_dep["mesure"].replace("\.", "", regex=True,inplace=True)
df_dep.mesure = df_dep.mesure.apply(lambda x: clean_str(x))

df_dep.rename(columns={"effect_id":"indicateur"}, inplace=True)
df_dep.indicateur = df_dep.indicateur.str.strip()
# traduit les mois dans la colonne date
df_dep.Date = df_dep.Date.replace(month_dict, regex=True)

df_dep.valeur.replace({",": "."}, regex=True, inplace=True)
df_dep.valeur = df_dep.valeur.astype(float)

In [ ]:
df_dep["short_indic"] = df_dep.indicateur.apply(lambda x: x.split("-")[0].strip())
df_dep.short_indic = df_dep.short_indic.apply(lambda x: clean_str(x))

In [ ]:
dict_indicateur = {'Nombre de repas servis dans les restaurants universitaires au tarif à 1€' : "Nombre de repas servis",
 'Montant cumulé de l’investissement total ainsi déclenché' : "Montant cumumé de l'investissement total",
 'Nombre d’entreprises bénéficiaires du dispositif' : "Nombre d'entreprises bénéficiaires",
 'Nombre de bâtiments Etat dont le marché de rénovation est notifié' : 'Nombre de bâtiments dont le marché de rénovation est notifié',
 'Nombre de projets incluant une transformation de la ligne de production pour réduire son impact environnemental' : 'Nombre de projets',
 'Nombre d\'exploitations certifiées "haute valeur environnementale"' : 'Nombre d’exploitations certifiées',
 'Emissions de gaz à effet de serre évitées sur la durée de vie des équipements' : 'Emissions de gaz à effet de serre évitées',
 'Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables' : 'Nombre de bonus octroyés à des véhicules électriques',
 "Quantité de matières plastiques évitées ou dont le recyclage ou l'intégration a été soutenue" : 'Quantité de matières plastiques évitées',
 'Montant total des travaux associés aux dossiers validés' : 'Montant total des travaux',
 'Nombre de nouveaux projets (nouvelle ligne, extension de ligne et pôle)' : 'Nombre de nouveaux projets',
 'Montant de l’investissement total déclenché' : 'Montant de l’investissement total',
 'Nombre de projets de tourisme durable financés' : 'Nombre de projets de tourisme durable financés',
 'Nombre de projets de rénovation de cathédrales et de monuments nationaux initiés' : 'Nombre de projets de rénovation',
 'Montant total investi pour la rénovation de monuments historiques appartenant aux collectivités territoriales' :'Montant total investi pour la rénovation',
 'Nombre de projets de rénovation de monuments historiques appartenant aux collectivités territoriales bénéficiaires initiés' : 'Nombre de projets de rénovation',
 "Nombre de contrats de professionnalisation bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats de professionnalisation',
 "Nombre de contrats d'apprentissage bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats d’apprentissage',
 "Nombre d'aides à l'embauche des travailleurs handicapés": "Nombre d'aides à l'embauche des travailleurs handicapés",
 'Nombre de projets locaux soutenus  (rénovation, extension, création de lignes' :'Nombre de projets locaux soutenus'
                  }

In [ ]:
df_dep.short_indic = df_dep.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)

In [ ]:
[x for x in list(df_dep.short_indic.unique()) if len(x) >40]

In [ ]:
dict_mesures  = {
 "Appels à projets pour les projets d'efficacité énergétique de grande ampleur et appels à manifestation d'intérêt sur les projets qui proposent une évolution profonde des procédés industriels": "AAP et AMI Efficacité énergétique",
 "CIE jeunes" : "Contrats Initiatives Emploi (CIE) Jeunes",
 'France Num' : 'France Num : aide à la numérisation des TPE,PME,ETI',
 'Guichet efficacité énergétique dans industrie' : 'Guichet efficacité énergétique',
 "Modernisation de la filière (BPI)" : "AAP industrie : Modernisation des filières auto et aéro",
 "PEC jeunes": "Parcours emploi compétences (PEC) Jeunes",
 'Relocalisation : soutien aux projets industriels dans les territoires' : 'AAP Industrie : Soutien aux projets industriels territoires',
 'Relocalisation : sécurisation des approvisionnements critiques' : 'AAP Industrie : Sécurisation approvisionnements critiques',
 'Renforcement des subventions de Business France (chèque export, chèque VIE)' : 'Renforcement subventions Business France',
 "Soutien à la modernisation industrielle et renforcement des compétences dans la filière nucléaire" : "AAP industrie : modernisation industrielle et renforcement des compétences dans la filière nucléaire",
 'Soutien à la recherche aéronautique civil' :'Soutien recherche aéronautique civil',
 'Rénovation bâtiments Etats' : 'Rénovation des bâtiments Etats (marchés notifiés)'
}

In [ ]:
df_dep["short_mesure"] = df_dep.mesure.apply(lambda x: dict_mesures[x] if x in dict_mesures else x)

In [ ]:
taxo_dep_df = pd.read_csv('refs/taxo_deps.csv', dtype={'dep':str, 'reg':str})
taxo_dep_df['dep'] = taxo_dep_df['dep'].apply(lambda x: x.zfill(2))
taxo_dep_df['reg'] = taxo_dep_df['reg'].apply(lambda x: x.zfill(2))
dep_list = list(taxo_dep_df['dep'].unique())
taxo_dep_df

In [ ]:
taxo_reg_df = pd.read_csv('refs/taxo_regions.csv', dtype={'reg':str})
taxo_reg_df['reg'] = taxo_reg_df['reg'].apply(lambda x: x.zfill(2))
reg_list = list(taxo_reg_df['reg'].unique())
taxo_reg_df

In [ ]:
d = df_dep.merge(taxo_dep_df[["dep", "reg", "libelle"]], how="left", left_on="dep_code", right_on="dep")
d.isnull().sum()

In [ ]:
df_dep_enr = df_dep.merge(taxo_dep_df[["dep", "reg", "libelle"]], 
                          how="left", left_on="dep_code", right_on="dep")\
                   .merge(taxo_reg_df[["reg", "nccenr"]], how="left", left_on="reg", right_on="reg")
df_dep_enr.drop(columns=["tree_node_name", "structure_name"], inplace=True)
#df_dep_enr = df_dep_enr.loc[~df_dep_enr.dep.isna(), :]
#df_dep_enr = df_dep_enr[df_dep_enr['departement'] != 'Etranger']
#assert df_dep_enr['dep'].isnull().sum() == 0
df_dep_enr.rename(columns={"nccenr":"region"}, inplace=True)
df_dep_enr.drop(['dep_code'], axis=1, inplace=True)  # Supprime code dep utilisé pour la jointure

assert df_dep_enr.shape[0] == df_dep.shape[0]
df_dep_enr.shape

In [ ]:
df_dep_enr.isnull().sum()

In [ ]:
#df_dep_enr[df_dep_enr['departement'] == 'Etranger']

In [ ]:
#df_dep_enr[df_dep_enr['dep'].isnull()]

In [ ]:
df_dep_enr.to_csv("pp_dep.csv", sep=";")

In [ ]:
tmp = df.loc[
    (~df.period_month_tri.isin(forbidden_period_value)) &
    (df.state_id == 'Valeur Actuelle') &
    (~df.valeur.isna())].copy()

In [ ]:
tmp["mesure"] = tmp["tree_node_name"].apply(lambda x: x.split("/")[1].strip() if '/' in x else x)

In [ ]:
month = 'Feb'
mesure = 'Service civique'
indic = 'Nombre d’entrées en service civique - SCI'

target = tmp[(tmp['structure_name'] == 'Département') & (tmp['mesure'] == mesure) & (tmp['effect_id'] == indic) & (tmp['period_month_tri'] == month)]
target

In [ ]:
df_dep[(df_dep['departement'] == 'Etranger') & (df_dep['structure_name'] == 'Département') & (df_dep['mesure'] == mesure) & (df_dep['indicateur'] == indic) & (df_dep['period_month_tri'] == month)]

In [ ]:
target['valeur'].sum()

In [ ]:
target['valeur'].sum() - 492